In [94]:
"""Readme :  pip install python-telegram-bot --upgrade"""

import logging
from typing import Dict

from telegram import ReplyKeyboardMarkup, Update, ReplyKeyboardRemove
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
TOKEN='5389959641:AAFzapxb0HqNfNtEKCBrpVJzTx5AexVX5Bk'

# State definitions for top level conversation
CHOOSING,QUERY_PLOT = map(chr, range(2))
# Shortcut for ConversationHandler.END
END = ConversationHandler.END

reply_keyboard = [
    [ 'Search movie by plot events','Recommend movies by similarity'],]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)



def start(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
    update.message.reply_text(
        
        "Hi  ! I'm Moviezam! I know everything about movies! \n" 
        "Do you want to search for movies with a particular plot event? \n" 
        "Can i suggest you similar movies to the ones you like?\n" 
        "Choose on of the options below, I start searching for you\n"
         "To abort, simply type /stop.",
        reply_markup=markup,
    )

    return CHOOSING

def answer_recommend(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text('Please write the title of a movie that you like, i will suggest you 5 similar movies')

    return TYPING_REPLY

def answer_plot(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text('Please write some plot events or details and i will show you some related movies')

    return QUERY_PLOT

def stop(update: Update, context: CallbackContext) -> int:
    """End Conversation by command."""
    update.message.reply_text('Okay, bye.')

    return END

def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    
    text_to_search=update.message.text
    df=search_plot_similarity_adjusted(text=text_to_search,encoding_method=word2vec_encoding_query,model=model_w2v,encoded_corpus=X_word2vec,dataframe=data_movielens_final_new,topk=11)
    df_lista=df['Links'].tolist()
    primo_film=df_lista[0]
    update.message.reply_text(primo_film)
    
    return END


def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(TOKEN)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [
                MessageHandler(
                    Filters.regex('^(Search movie by plot events)$'), answer_plot   ),
                MessageHandler(Filters.regex('^Recommend movies by similarity$'), answer_recommend),
            ],
            
            
            QUERY_PLOT: [MessageHandler(Filters.text & ~Filters.command, echo)],
       
        },
        
        fallbacks=[CommandHandler('stop', stop)],
    )

    dispatcher.add_handler(conv_handler)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()


2022-04-24 23:49:01,700 - apscheduler.scheduler - INFO - Scheduler started
C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\2254915620.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity']=np.sort(cosineSimilarities)[-topk*20:].tolist()
C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\2254915620.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_score']=(df['similarity']*0.87)+(df['votes_score']*0.13)
C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\2254915620.py:6: Settin

In [54]:
import pandas as pd
from gensim.models import Word2Vec
import pickle
import emoji
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import simplemma
import string
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
data_movielens_final_new=pd.read_excel('C:/Users/cioffil/Desktop/DS/Github/Moviezam/Moviezam/data/movielens/data_movielens_final_new.xlsx')

In [86]:

data_movielens_final_new['zeros']=np.where(data_movielens_final_new['ID_Length']==7, '', np.where(data_movielens_final_new['ID_Length']==6,'0',np.where(data_movielens_final_new['ID_Length']==5,'00',np.where(data_movielens_final_new['ID_Length']==4,'000',np.where(data_movielens_final_new['ID_Length']==3,'0000',np.where(data_movielens_final_new['ID_Length']==2,'00000','000000'))))))
data_movielens_final_new['Links'] = 'https://www.imdb.com/title/tt'+data_movielens_final_new['zeros']+data_movielens_final_new['ID'].astype(str)
data_movielens_final_new.tail()

,Unnamed: 0,ID,text,text_clean,Title,original_title,votes_score,year,languages,overview,plot_outline,Synopsis,ID_Length,Links,zeros
134999,134999,1003009,En su recorrido por el interior de la Casona E...,en su recorrido por el interior de la casona e...,Legend Quest: The Legend of La Nahuala,La leyenda de la Nahuala,0.419209,2007.0,['Spanish'],NaN,The movie takes place in the year 1807 in the ...,Sinópsis Corre el año de 1807 en la Ciudad de ...,7,https://www.imdb.com/title/tt1003009,
135000,135000,1003009,"Among these stories is ""The Legend of the Nahu...",among story legend nahuala accord old abandon ...,Legend Quest: The Legend of La Nahuala,La leyenda de la Nahuala,0.419209,2007.0,['Spanish'],NaN,The movie takes place in the year 1807 in the ...,Sinópsis Corre el año de 1807 en la Ciudad de ...,7,https://www.imdb.com/title/tt1003009,
135001,135001,1003009,"In his journey inside the bewitched house, Leo...",journey inside bewitch house Leo face danger s...,Legend Quest: The Legend of La Nahuala,La leyenda de la Nahuala,0.419209,2007.0,['Spanish'],NaN,The movie takes place in the year 1807 in the ...,Sinópsis Corre el año de 1807 en la Ciudad de ...,7,https://www.imdb.com/title/tt1003009,
135002,135002,5606664,She finds what looks like filing cabinets and ...,find look like filing cabinet try dig abra ant...,Doctor Sleep,Doctor Sleep,0.820093,2019.0,['English'],NaN,"On highways across America, a tribe of people ...","Florida, 1981 A little girl named Violet Hanse...",7,https://www.imdb.com/title/tt5606664,
135003,135003,5213870,A young New York journalist returns to the idy...,young new york journalist return idyllic centr...,Pray for Rain,Pray for Rain,0.469513,2017.0,['English'],NaN,When whip-smart New York City reporter Emma Ga...,A young New York journalist returns to the idy...,7,https://www.imdb.com/title/tt5213870,


In [22]:
# upload w2v
model_w2v = pd.read_pickle('C:/Users/cioffil/Desktop/DS/Github/Moviezam/Moviezam/pickle_data/pickle_w2v')
X_word2vec = pd.read_pickle('C:/Users/cioffil/Desktop/DS/Github/Moviezam/Moviezam/pickle_data/pickle_w2v_vectors')

In [87]:
def search_plot_similarity_adjusted(text,encoding_method,encoded_corpus,dataframe,topk,model=None,vectorizer=None,features=None):
	TOKEN_LEMMA = [clean_text_lemma(text)]
	space_query=encoding_method(TOKEN_LEMMA,model)
	cosineSimilarities = cosine_similarity(space_query, encoded_corpus).flatten()
	df=dataframe.iloc[np.argsort(cosineSimilarities)[-topk*20:].tolist() ]
	df['similarity']=np.sort(cosineSimilarities)[-topk*20:].tolist() 
	df['final_score']=(df['similarity']*0.87)+(df['votes_score']*0.13)
	df=df.sort_values(['final_score'], ascending=False)
	df=df.groupby('Title').first().reset_index()
	df=df.nlargest(topk,'final_score')
    
	df=df[['Title','Links']]

	return df
def word2vec_encoding_query(text,model_w2v,percentage_of_vocabulary_to_use=None, min_count=5):
    size=300
    corpus = []
    tokenizer = WordPunctTokenizer()
    for sent in text:
        corpus.append(tokenizer.tokenize(sent.lower().translate(str.maketrans('', '', string.punctuation))))
    vec = np.zeros(size).reshape((1, size))
    wordvec_arrays = np.zeros((len(corpus), 300)) 
    for i in range(len(corpus)):
        count = 0
        for word in corpus[i]:
            try:
                vec += model_w2v.wv[word].reshape((1, size)) 
                count += 1.
            except KeyError:  # handling the case where the token is not in vocabulary
                continue
        if count != 0:
            vec /= count
        wordvec_arrays[i,:] = vec
    
    return   wordvec_arrays 
stopwords_en = stopwords.words('english')
langdata = simplemma.load_data('en')

def clean_text_lemma(text):
    if type(text) == float:
        return ""
    temp = text.lower() 
    temp = re.sub("'", " ", temp) # in inglese rimuove le parole abbreviate ma anche in italiano serve a staccare le parola con apostrofo
    temp=emoji.demojize(temp, language='en')
    words = temp.split()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9èéàùòì]"," ", temp)
    temp = re.sub(" \d+", " ", temp)
    temp = re.sub("^[0-9]", " ", temp)
    #temp = temp.split()
    temp = WordPunctTokenizer().tokenize(temp)
    temp = [simplemma.lemmatize(w, langdata) for w in temp if not w in stopwords_en]
    temp = " ".join(word for word in temp)
    
    return temp


In [93]:
text_to_search='american young boy does time travel and meet his mom and dad but his mother fall in love with him with the help of a scientist will be sent back in the present'

df=search_plot_similarity_adjusted(text=text_to_search,encoding_method=word2vec_encoding_query,model=model_w2v,encoded_corpus=X_word2vec,dataframe=data_movielens_final_new,topk=11)
df_lista=df['Links'].tolist()
primo_film=df_lista[0]

C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\2254915620.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity']=np.sort(cosineSimilarities)[-topk*20:].tolist()
C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\2254915620.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_score']=(df['similarity']*0.87)+(df['votes_score']*0.13)


['https://www.imdb.com/title/tt0212338',
 'https://www.imdb.com/title/tt0088763',
 'https://www.imdb.com/title/tt2194499',
 'https://www.imdb.com/title/tt0332280',
 'https://www.imdb.com/title/tt0108160',
 'https://www.imdb.com/title/tt4824302',
 'https://www.imdb.com/title/tt0838283',
 'https://www.imdb.com/title/tt0120783',
 'https://www.imdb.com/title/tt0050986',
 'https://www.imdb.com/title/tt0832266',
 'https://www.imdb.com/title/tt0114887']